In [3]:
import requests
from dotenv import load_dotenv


In [4]:
# GET API KEY FROM THE .env file
load_dotenv()
API_KEY = os.getenv('API_KEY')

### gettime request - checks for local clock synced with CTA

In [5]:
api_url = f'http://www.ctabustracker.com/bustime/api/v2/gettime?key={API_KEY}'

response = requests.get(api_url)
response.headers["Content-Type"]
response.text

### getvehicles request by route(s) or vehicle(s)

In [8]:
# Must specify a route or list of routes via &rt=20,22 (to get routes 20 and 22)
# Alternatively, you could specify a vehicle or list of vehicles instead
# Must specify &format=json, otherwise format will be html

api_url = f'http://www.ctabustracker.com/bustime/api/v2/getvehicles?key={API_KEY}&rt=20,22&format=json'

response = requests.get(api_url)
test_vehicles_20_and_22 = response.json()
test_vehicles_20_and_22

### getroutes request

In [10]:
# Must specify &format=json, otherwise format will be html

api_url = f'http://www.ctabustracker.com/bustime/api/v2/getroutes?key={API_KEY}&format=json'

response = requests.get(api_url)
routes = response.json()
routes

### getdirections request

In [17]:
# Must specify a single route: &rt=20 for Route 20, for example

api_url = f'http://www.ctabustracker.com/bustime/api/v2/getdirections?key={API_KEY}&rt=20&format=json'

response = requests.get(api_url)
directions_rt_20 = response.json()
directions_rt_20

{'bustime-response': {'directions': [{'dir': 'Eastbound'},
   {'dir': 'Westbound'}]}}